# Example of how to use the Extreme Correction methodology for Hs

In this notebook, the Extreme Correction methodology is applied to the Hs data using the class created for this purpose

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import sys
import os

# Add the 'src' folder to sys.path
sys.path.append(os.path.abspath('../../src'))

# Check the working directory
os.chdir("../../")
print(os.getcwd())

# Clase creada
from extremecorrection import Gev_ExtremeCorrection

c:\Users\user\Victor\Trabajo\ExtremeCorrection


#### Load historical data

In [2]:
df_hist = pd.read_table("Data/DatosHs_SS/Hs_SS_PCs_FWT_LWT_Precip_Cantabria_reg_guided.txt")

In [3]:
df_hist.head()

,yyyy,mm,dd,Hs,Surge,PC0,PC1,PC2,PC3,PC4,...,PC67_SS,PC68_SS,PC69_SS,PC70_SS,PC71_SS,PC72_SS,PC73_SS,PC74_SS,LWT,precip
0,1979,1,1,1.023583,-0.059125,51.681534,30.356916,-6.727809,-19.387419,0.232885,...,-0.001808,-0.003190,0.000548,-0.006154,-0.008700,0.000939,-0.004016,0.000593,0,4.104891
1,1979,1,2,1.341333,-0.107542,40.511876,38.299683,17.834673,-24.315038,3.024192,...,0.000203,-0.001587,-0.000242,0.000403,0.000597,-0.001703,0.001418,-0.000744,3,2.854573
2,1979,1,3,1.855333,0.128542,17.988358,37.808518,12.285333,-0.991782,15.546282,...,-0.001919,-0.006954,0.004526,-0.000502,-0.003448,0.002078,0.008224,-0.001982,14,5.023113
3,1979,1,4,4.830250,0.215208,5.815724,31.481725,-14.426114,15.671999,17.975393,...,-0.002443,0.001002,0.003585,0.001733,-0.001514,0.001234,-0.000144,0.000159,6,0.000000
4,1979,1,5,2.902167,0.056542,-5.434250,18.093675,-15.427994,12.982450,7.758927,...,0.001845,-0.002186,0.000825,0.001760,0.000370,-0.003263,-0.000107,-0.000279,5,4.834372


#### Load simulated data

In [4]:
ds_sim = xr.open_dataset("Data/DatosHs_SS/FWT_Hs_sim_new.nc")

Transform dataset into dataframe

In [5]:
ds_sim

<xarray.Dataset>
Dimensions:  (n_sim: 100, time: 36526)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2100-01-01
  * n_sim    (n_sim) int64 0 1 10 11 12 13 14 15 16 ... 92 93 94 95 96 97 98 99
Data variables:
    Hs       (n_sim, time) float64 ...
    FWT      (n_sim, time) int64 ...

In [6]:
nsim = ds_sim.n_sim.values

ss_simulated_data = np.array([])
year_simulated = np.array([])
month_simulated = np.array([])
day_simulated = np.array([])
wt_simulated_data = np.array([])
for i in range(len(nsim)):
    ss_simulated_data = np.concatenate([ss_simulated_data, ds_sim.Hs[i].values[:-1]])
    year_simulated = np.concatenate([year_simulated, ds_sim.time.values[:-1].astype('datetime64[Y]').astype(int) + 1970  + 100*i])
    month_simulated = np.concatenate([month_simulated, ds_sim.time.values[:-1].astype('datetime64[M]').astype(int) % 12 + 1])
    day_simulated = np.concatenate([day_simulated, (ds_sim.time.values[:-1].astype('datetime64[D]') - ds_sim.time.values[:-1].astype('datetime64[M]')).astype(int) + 1])
    wt_simulated_data = np.concatenate([wt_simulated_data, ds_sim.FWT[i,:-1].values])   # FWT indices: (n_sim, time)

# Cambiar el indice del WT para que se asemeje a los datos historicos (rango WT0-WT35, nuevos datos rango WT1-WT36)
wt_simulated_data = wt_simulated_data -1

# Dataframe with the simulated data
df_sim = pd.DataFrame({'Hs': ss_simulated_data,
                       'yyyy': year_simulated,
                        'mm': month_simulated,
                        'dd': day_simulated,
                        'wt': wt_simulated_data})

In [7]:
df_sim.head()

,Hs,yyyy,mm,dd,wt
0,1.827589,2000.0,1.0,1.0,5.0
1,2.096124,2000.0,1.0,2.0,12.0
2,1.044530,2000.0,1.0,3.0,12.0
3,3.096138,2000.0,1.0,4.0,15.0
4,1.366497,2000.0,1.0,5.0,25.0


#### Correction

In [8]:
config = {
    'var': 'Hs',
    'yyyy_var': 'yyyy',
    'mm_var': 'mm',
    'dd_var': 'dd',
    'freq': 365.25,
    'folder': 'Figures/Hs_Complete'
}

Initialize the class

In [9]:
extr_corr = Gev_ExtremeCorrection(
    data = df_hist,
    config = config,
    conf = 0.95
)

Apply historical correction

In [10]:
extr_corr.apply_correction(fit_diag=True)   # fit_diag = True, to plot the diagnostics plots (QQ-Plot and PP-plot)
extr_corr.return_period_plot()              # Default to plot only the no corrected data 

In [11]:
extr_corr.gev_parameters    # shape parameter is in negative

[6.918381550594482, 0.8294173278481032, -0.03662350897345275]

Apply simulated correction

In [ ]:
extr_corr.apply_correction_sim(simulated_data = df_sim)   # Apply the correction in the simulated data
extr_corr.sim_return_period_plot()                        # Default to plot the corrected and no corrected data
extr_corr.interval_sim_return_period_plot()               # Comparative of no corrected vs corrected simulated data in the historical range
extr_corr.time_series_plot(sim=False)                     # Time series of both corrected and no-corrected data

Dataframe with the corrected simulation

In [11]:
new_sim_data = extr_corr.sim_pit_data_corrected
new_df_sim = df_sim.copy()
new_df_sim['Hs'] = new_sim_data